<b>1. Load libraries

In [24]:
import requests # library to handle requests
import pandas as pd # library for data analsysis

!conda install geopandas
import geopandas as gpd


import numpy as np # library to handle data in a vectorized manner
import json

#!conda config --add channels conda-forge

!conda install shapely
from shapely.geometry import shape, Point

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
#from IPython.display import Image 
#from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


<b> 2. Load Moscow districts data (geojson)</b>

In [33]:

url = 'http://gis-lab.info/data/mos-adm/ao.geojson'
#response = requests.get(url)
#data = response.json()
#df = pd.io.json.json_normalize(data['features'])
gdf = gpd.read_file(url)
gdf.head()

df = pd.DataFrame(gdf.iloc[:,:-1].values, columns = list(gdf.columns.values)[:-1] )
df.head()

,NAME,OKATO,ABBREV
0,Троицкий,45298000,Троицкий
1,Новомосковский,45297000,Новомосковский
2,Зеленоградский,45272000,ЗелАО
3,Юго-Западный,45293000,ЮЗАО
4,Юго-Восточный,45290000,ЮВАО


In [7]:
type(df)

geopandas.geodataframe.GeoDataFrame

In [34]:
df['name_length'] = df['NAME'].str.len()
df.sort_values('name_length', ascending=False, inplace=True)
df.head(12)

,NAME,OKATO,ABBREV,name_length
8,Северо-Восточный,45280000,СВАО,16
7,Северо-Западный,45283000,СЗАО,15
1,Новомосковский,45297000,Новомосковский,14
2,Зеленоградский,45272000,ЗелАО,14
4,Юго-Восточный,45290000,ЮВАО,13
3,Юго-Западный,45293000,ЮЗАО,12
5,Центральный,45286000,ЦАО,11
10,Восточный,45263000,ВАО,9
0,Троицкий,45298000,Троицкий,8
6,Северный,45277000,САО,8


<b> 3. Load restaurants/cafes data for Moscow</b>

In [10]:

url = 'https://raw.githubusercontent.com/RunnerTony/CoureraProjectsRepo/master/moscow_eatout_json_utf8.txt'

results = requests.get(url).json()

df_cafes = json_normalize(results)
#df_cafes=pd.read_json'https://raw.githubusercontent.com/RunnerTony/CoureraProjectsRepo/master/moscow_eatout_json', encoding='WINDOWS-1251')
df_cafes.head()

,Address,AdmArea,District,ID,IsNetObject,Latitude_WGS84,Longitude_WGS84,Name,OperatingCompany,PublicPhone,SeatsCount,SocialPrivileges,TypeObject,geoData.coordinates,geoData.type,global_id
0,"город Москва, улица Егора Абакумова, дом 9",Северо-Восточный административный округ,Ярославский район,00151635,нет,55.8790015313033730,37.7145650004360390,СМЕТАНА,NaN,[{'PublicPhone': '(499) 183-14-10'}],48,нет,кафе,"[37.71456500043604, 55.87900153130337]",Point,637376221
1,"город Москва, улица Талалихина, дом 2/1, корпус 1",Центральный административный округ,Таганский район,000077874,нет,55.7382386551547000,37.6733061300344000,Родник,NaN,[{'PublicPhone': '(495) 676-55-35'}],35,нет,кафе,"[37.6733061300344, 55.7382386551547]",Point,637376331
2,"город Москва, Абельмановская улица, дом 6",Центральный административный округ,Таганский район,000024309,нет,55.7355114718314000,37.6696475969381000,Кафе «Академия»,NaN,[{'PublicPhone': '(495) 662-30-10'}],95,нет,кафе,"[37.6696475969381, 55.7355114718314]",Point,637376349
3,"город Москва, Абрамцевская улица, дом 9, корпус 1",Северо-Восточный административный округ,район Лианозово,00119365,нет,55.9040863698490400,37.5723061316711200,Кафе «Вишневая метель»,NaN,[{'PublicPhone': '(499) 200-00-22'}],50,нет,кафе,"[37.57230613167112, 55.90408636984904]",Point,637376403
4,"город Москва, Угличская улица, дом 17",Северо-Восточный административный округ,район Лианозово,000027429,нет,55.9040188006692200,37.5669400000000020,ГБОУ «Школа № 1430 имени Героя Социалистическо...,NaN,[{'PublicPhone': '(499) 908-06-15'}],240,нет,столовая,"[37.56694, 55.90401880066922]",Point,637376480


In [42]:
df_cafes.shape

(16299, 16)

In [11]:
df_cafes['TypeObject'].value_counts()

кафе                                 6464
столовая                             2439
ресторан                             2318
предприятие быстрого обслуживания    2181
бар                                  1000
буфет                                 574
кафетерий                             553
закусочная                            434
магазин (отдел кулинарии)             336
Name: TypeObject, dtype: int64

In [12]:
df_cafes = df_cafes[df_cafes.SeatsCount != 0]
df_cafes.shape

(14266, 16)

In [13]:
df_cafes['TypeObject'].value_counts()

кафе                                 6052
столовая                             2418
ресторан                             2277
предприятие быстрого обслуживания    1291
бар                                   964
буфет                                 489
кафетерий                             412
закусочная                            211
магазин (отдел кулинарии)             152
Name: TypeObject, dtype: int64

<b> 4. Aggregate seats count data and merge it to the geojson dataframe (df) </b>

In [35]:
df_tmp = df_cafes
Scount = []
print('Seats count by district:')
for distr in df['NAME'].values:
    df2 = df_tmp[df_tmp['AdmArea'].str.contains(distr)]
    df_tmp = df_tmp[df_tmp['AdmArea'].str.contains(distr)==False]
    s = df2['SeatsCount'].sum()
    print(distr,': ', s)
    Scount.append(s)
    
df['SeatsCount']=Scount

Seats count by district:
Северо-Восточный :  90083
Северо-Западный :  44228
Новомосковский :  17075
Зеленоградский :  17159
Юго-Восточный :  65867
Юго-Западный :  66151
Центральный :  245833
Восточный :  74862
Троицкий :  6901
Северный :  72842
Западный :  91702
Южный :  103701


In [36]:
df.head()

,NAME,OKATO,ABBREV,name_length,SeatsCount
8,Северо-Восточный,45280000,СВАО,16,90083
7,Северо-Западный,45283000,СЗАО,15,44228
1,Новомосковский,45297000,Новомосковский,14,17075
2,Зеленоградский,45272000,ЗелАО,14,17159
4,Юго-Восточный,45290000,ЮВАО,13,65867


<b>5. Load population by district and merge that into the geojson dataframe (df)</b>

In [16]:
df_pop=pd.read_csv("https://raw.githubusercontent.com/RunnerTony/CoureraProjectsRepo/master/moscow_population_v2.csv", encoding='WINDOWS-1251')
df_pop.shape

(176, 4)

In [17]:
df_pop['name_length'] = df_pop['Муниципальное образование'].str.len()
df_pop.sort_values('name_length', ascending=False, inplace=True)
df_pop.head()

,Муниципальное образование,Все население,в т.ч. городское,в т.ч. сельское,name_length
60,в том числе муниципальные образования Северо-В...,NaN,NaN,NaN,63
79,в том числе муниципальные образования Северо-З...,NaN,NaN,NaN,62
147,в том числе муниципальные образования Новомоск...,NaN,NaN,NaN,61
35,в том числе муниципальные образования Зеленогр...,NaN,NaN,NaN,61
101,в том числе муниципальные образования Юго-Вост...,NaN,NaN,NaN,60


In [37]:
df_tmp = df_pop
Pcount = []
print('Population by district:')
for distr in df['NAME'].values:
    df2 = df_tmp[df_tmp['Муниципальное образование'].str.contains(distr)]
    df_tmp = df_tmp[df_tmp['Муниципальное образование'].str.contains(distr)==False]
    s = df2['Все население'].sum()
    print(distr,': ', s)
    Pcount.append(s)
    
df['Population']=Pcount
df.head()

Population by district:
Северо-Восточный :  1 402 928
Северо-Западный :  979 614
Новомосковский :  183 591
Зеленоградский :  232 489
Юго-Восточный :  1 363 859
Юго-Западный :  1 414 510
Центральный :  760 690
Восточный :  1 495 835
Троицкий :  108 063
Северный :  1 151 160
Западный :  1 344 044
Южный :  1 760 813


,NAME,OKATO,ABBREV,name_length,SeatsCount,Population
8,Северо-Восточный,45280000,СВАО,16,90083,1 402 928
7,Северо-Западный,45283000,СЗАО,15,44228,979 614
1,Новомосковский,45297000,Новомосковский,14,17075,183 591
2,Зеленоградский,45272000,ЗелАО,14,17159,232 489
4,Юго-Восточный,45290000,ЮВАО,13,65867,1 363 859


In [38]:
#df['Saturation']=df['SeatsCount']/df['Population']
#df.head()
#if type(df['Population'].values[1]) == object:
#    df['Population'] = df['Population'].str.replace(' ','')
#    df['Population']=pd.to_numeric(df['Population'])

df['Population'] = df['Population'].str.replace(' ','')
df['Population']=pd.to_numeric(df['Population'])    
df['Saturation']=1000*df['SeatsCount']/df['Population']
df.head()

,NAME,OKATO,ABBREV,name_length,SeatsCount,Population,Saturation
8,Северо-Восточный,45280000,СВАО,16,90083,1402928,64.210708
7,Северо-Западный,45283000,СЗАО,15,44228,979614,45.148395
1,Новомосковский,45297000,Новомосковский,14,17075,183591,93.005648
2,Зеленоградский,45272000,ЗелАО,14,17159,232489,73.805642
4,Юго-Восточный,45290000,ЮВАО,13,65867,1363859,48.294582


In [22]:
df.head(12)

,NAME,OKATO,ABBREV,geometry,name_length,SeatsCount,Population,Saturation
8,Северо-Восточный,45280000,СВАО,"POLYGON ((37.51948 55.94176, 37.52004 55.94185...",16,90083,1402928,64.210708
7,Северо-Западный,45283000,СЗАО,"POLYGON ((37.33204 55.84595, 37.33205 55.84598...",15,44228,979614,45.148395
1,Новомосковский,45297000,Новомосковский,"MULTIPOLYGON (((37.08697 55.59036, 37.09492 55...",14,17075,183591,93.005648
2,Зеленоградский,45272000,ЗелАО,"POLYGON ((37.13160 56.01645, 37.13266 56.01678...",14,17159,232489,73.805642
4,Юго-Восточный,45290000,ЮВАО,"MULTIPOLYGON (((37.66069 55.73070, 37.66082 55...",13,65867,1363859,48.294582
3,Юго-Западный,45293000,ЮЗАО,"POLYGON ((37.45572 55.63705, 37.46372 55.64070...",12,66151,1414510,46.766018
5,Центральный,45286000,ЦАО,"POLYGON ((37.51393 55.75842, 37.51567 55.75909...",11,245833,760690,323.171068
10,Восточный,45263000,ВАО,"MULTIPOLYGON (((37.65031 55.79292, 37.65037 55...",9,74862,1495835,50.046964
0,Троицкий,45298000,Троицкий,"MULTIPOLYGON (((36.80310 55.44083, 36.80319 55...",8,6901,108063,63.860896
6,Северный,45277000,САО,"MULTIPOLYGON (((37.43648 55.87958, 37.43749 55...",8,72842,1151160,63.277042


<b> 6. Create a "heat map" of districts based on seat counts data (folium choropleth map)</b>

In [28]:
#df['geometry'].crs = "EPSG:4326"
#df.head()

,NAME,OKATO,ABBREV,geometry,name_length,SeatsCount,Population,Saturation
8,Северо-Восточный,45280000,СВАО,"POLYGON ((37.51948 55.94176, 37.52004 55.94185...",16,90083,1402928,64.210708
7,Северо-Западный,45283000,СЗАО,"POLYGON ((37.33204 55.84595, 37.33205 55.84598...",15,44228,979614,45.148395
1,Новомосковский,45297000,Новомосковский,"MULTIPOLYGON (((37.08697 55.59036, 37.09492 55...",14,17075,183591,93.005648
2,Зеленоградский,45272000,ЗелАО,"POLYGON ((37.13160 56.01645, 37.13266 56.01678...",14,17159,232489,73.805642
4,Юго-Восточный,45290000,ЮВАО,"MULTIPOLYGON (((37.66069 55.73070, 37.66082 55...",13,65867,1363859,48.294582


In [44]:
seats_map = folium.Map(location=[55.709, 37.627], zoom_start=11)

#seats_map.choropleth(geo_data=gdf, data=df,columns=['NAME', 'Saturation'],fill_color = 'BuPu')
                     #key_on='feature.properties.PCON13CD',
    
#seats_map.choropleth(geo_data=df, name='choropleth', data=df, columns=['NAME', 'Saturation'], key_on='feature.id', fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2, legend_name='Saturation (seats per 1000 persons)')

seats_map.choropleth(geo_data=gdf, name='choropleth', data=df, columns=['ABBREV', 'Saturation'], key_on='feature.id', fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2, legend_name='Saturation (seats per 1000 persons)')
folium.LayerControl().add_to(seats_map)


seats_map
